# Cross-Validation

In [ ]:
from sklearn.model_selection import StratifiedKFold
import gensim.models
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from tensorflow.keras.callbacks import EarlyStopping


kf = StratifiedKFold(5, shuffle=True, random_state=42) 

oos_y = []
oos_pred = []
fold = 0

# Must specify y StratifiedKFold for
for train, test in kf.split(X,df['Class']):  
    fold+=1
    print(f"Fold #{fold}")
        
    x_train1 = X[train]
    y_train1 = dummy_y[train]
    x_test1 = X[test]
    y_test1 = dummy_y[test]
    sentences1 = [row.split(' ') for row in x_train1]
    model1 = gensim.models.Word2Vec(sentences1, min_count=1,size= 100,workers=3, window =3, sg = 1)

    tokenizer1 = Tokenizer()
    tokenizer1.fit_on_texts(x_train1)


    sequences1 = tokenizer1.texts_to_sequences(x_train1)
    
    vocab_num1 = 0
    for i in sequences1:
        for j in i:
            if vocab_num1 < j:
                vocab_num1 = j
                
    x_train_seq1 = pad_sequences(sequences1, maxlen=max_len)
    
    sequences_val1 = tokenizer1.texts_to_sequences(x_test1)
    x_val_seq1 = pad_sequences(sequences_val1, maxlen=max_len)
    
    
    embeddings_index1 = {}
    for w in model1.wv.vocab.keys():
        embeddings_index1[w] = model1.wv[w]
        
    embedding_matrix1 = np.zeros((vocab_num1+1, 100))
    for word, i in tokenizer1.word_index.items():
        embedding_vector1 = embeddings_index1.get(word)
        if embedding_vector1 is not None:
            embedding_matrix1[i] = embedding_vector1
    
    from tensorflow.keras.callbacks import EarlyStopping

    classifier1 = Sequential()
    e1 = Embedding(vocab_num1+1, 100, weights=[embedding_matrix1], input_length=max_len, trainable=True)
    classifier1.add(e1)
    classifier1.add(Flatten())
    classifier1.add(Dense(256, activation='relu'))
    classifier1.add(Dense(nb_classes, activation='softmax'))
    classifier1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    monitor1 = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, 
            verbose=1, mode='auto', restore_best_weights=True)

    classifier1.fit(x_train_seq1, y_train1, validation_data=(x_val_seq1, y_test1), 
            callbacks=[monitor1], epochs=30, batch_size=32, verbose=1)
    
    
    pred = classifier1.predict(x_test1)
    
    oos_y.append(y_test1)
    # raw probabilities to chosen class (highest probability)
    pred = np.argmax(pred,axis=1) 
    oos_pred.append(pred)  

    # Measure this fold's accuracy
    y_compare = np.argmax(y_test1,axis=1) # For accuracy calculation
    score = metrics.accuracy_score(y_compare, pred)
    print(f"Fold score (accuracy): {score}")


In [ ]:
# Build the oos prediction list and calculate the error.
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
oos_y_compare = np.argmax(oos_y,axis=1) # For accuracy calculation

score = metrics.accuracy_score(oos_y_compare, oos_pred)
print(f"Final score (accuracy): {score}")    
    
# Write the cross-validated prediction
oos_y = pd.DataFrame(oos_y)
oos_pred = pd.DataFrame(oos_pred)
oosDF = pd.concat( [df, oos_y, oos_pred],axis=1 )
#oosDF.to_csv(filename_write,index=False)
    

In [ ]:
from sklearn.model_selection import StratifiedKFold
import gensim.models
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
SEED = 2000




# Keep a 10% holdout
x_main, x_holdout, y_main, y_holdout = train_test_split(
    X, Y, test_size=.02, random_state=SEED, stratify = Y)

encoded_Y_main = encoder.transform(y_main)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y_main = to_categorical(encoded_Y_main)



kf = StratifiedKFold(5, shuffle=True, random_state=42) 

oos_y = []
oos_pred = []
fold = 0

# Must specify y StratifiedKFold for
for train, test in kf.split(x_main, y_main):  
    fold+=1
    print(f"Fold #{fold}")
        
    x_train1 = x_main[train].astype(str)
    y_train1 = dummy_y_main[train]
    x_test1 = x_main[test]
    y_test1 = dummy_y_main[test]
    sentences1 = [row.split(' ') for row in x_train1]
    model1 = gensim.models.Word2Vec(sentences1, min_count=1,size= 100,workers=3, window =3, sg = 1)

    tokenizer1 = Tokenizer()
    tokenizer1.fit_on_texts(x_train1)


    sequences1 = tokenizer1.texts_to_sequences(x_train1)
    
    vocab_num1 = 0
    for i in sequences1:
        for j in i:
            if vocab_num1 < j:
                vocab_num1 = j
                
    x_train_seq1 = pad_sequences(sequences1, maxlen=max_len)
    
    sequences_val1 = tokenizer1.texts_to_sequences(x_test1)
    x_val_seq1 = pad_sequences(sequences_val1, maxlen=max_len)
    
    
    embeddings_index1 = {}
    for w in model1.wv.vocab.keys():
        embeddings_index1[w] = model1.wv[w]
        
    embedding_matrix1 = np.zeros((vocab_num1+1, 100))
    for word, i in tokenizer1.word_index.items():
        embedding_vector1 = embeddings_index1.get(word)
        if embedding_vector1 is not None:
            embedding_matrix1[i] = embedding_vector1
    
    from tensorflow.keras.callbacks import EarlyStopping

    classifier1 = Sequential()
    e1 = Embedding(vocab_num1+1, 100, weights=[embedding_matrix1], input_length=max_len, trainable=True)
    classifier1.add(e1)
    classifier1.add(Flatten())
    classifier1.add(Dense(256, activation='relu'))
    classifier1.add(Dense(nb_classes, activation='softmax'))
    classifier1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    monitor1 = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, 
            verbose=1, mode='auto', restore_best_weights=True)

    classifier1.fit(x_train_seq1, y_train1, validation_data=(x_val_seq1, y_test1), 
            callbacks=[monitor1], epochs=30, batch_size=32, verbose=1)
    
    
    pred = classifier1.predict(x_test1)
    
    oos_y.append(y_test1)
    # raw probabilities to chosen class (highest probability)
    pred = np.argmax(pred,axis=1) 
    oos_pred.append(pred)  

    # Measure this fold's accuracy
    y_compare = np.argmax(y_test1,axis=1) # For accuracy calculation
    score = metrics.accuracy_score(y_compare, pred)
    print(f"Fold score (accuracy): {score}")

    
    


In [ ]:
# Build the oos prediction list and calculate the error.
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
score = np.sqrt(metrics.mean_squared_error(oos_pred,oos_y))
print()
print(f"Cross-validated score (RMSE): {score}")    
    
# Write the cross-validated prediction (from the last neural network)
holdout_pred = model.predict(x_holdout)

score = np.sqrt(metrics.mean_squared_error(holdout_pred,y_holdout))
print(f"Holdout score (RMSE): {score}")